In [1]:
# default_exp secret
# hide

_FNAME='secret'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os

TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')
HERE = os.path.abspath('')

In [2]:
#export
import boto3
CLIENT = boto3.client('secretsmanager')

In [3]:
#export 
from scylla import run_container, set_port_name
import localstack_client.session

def run_localstack(version="latest", **kwargs):
    env = {'SERVICES':'secretsmanager'}
    additional_kwargs = {'environment': env}
    additional_kwargs.update(kwargs)
    #changing the port of localstack is hard and involves tinkering with the EDGE_PORT environment variable
    #just leave it be
    ports = {4566:4566}
    image = 'localstack/localstack:{}'.format(version)
    ls_container = run_container(image, ports=ports, block_until_port_available=4566,
                                   **additional_kwargs)
    ls_container = set_port_name(ls_container, 4566, 'entry')
    return ls_container    

def make_local_client():
    session = localstack_client.session.Session()
    return session.client('secretsmanager')

In [4]:
CONTAINER_NAME = 'localstack-vera-secret'
ls_container = run_localstack(container_name=CONTAINER_NAME)

In [13]:
#export
def get(name, client=None, fail_safely=False, **kwargs):
    '''
    Retrieves the secret with the given name
    '''
    client = client or CLIENT
    try:
        resp = client.get_secret_value(SecretId=name, **kwargs)
    except client.exceptions.ResourceNotFoundException:
        if fail_safely:
            return None
        raise    
    str_value = resp.get('SecretString')
    byte_value = resp.get('SecretBytes')
    return str_value or byte_value

def create(name, value, client=None, **kwargs):
    '''
    Create a new secret with Name=name
    If you want SecretBytes instead of SecretString
    set value=None and specify SecretBytes in kwargs
    '''
    client = client or CLIENT
    if value is not None:
        kwargs['SecretString'] = value
    
    resp = client.create_secret(Name=name, **kwargs)
    ok = resp['ResponseMetadata']['HTTPStatusCode']==200
    return ok, resp

def update(name, value, client=None, **kwargs):
    '''
    Update the value of a secret with SecretId=name
    '''
    client = client or CLIENT
    if value is not None:
        kwargs['SecretString'] = value
    
    resp = client.put_secret_value(SecretId=name, **kwargs)    
    ok = resp['ResponseMetadata']['HTTPStatusCode']==200
    return ok, resp

def delete(name, client=None, **kwargs):
    '''
    Schedule the secret with SecretId=name for deletion
    '''
    client = client or CLIENT
    resp = client.delete_secret(SecretId=name, **kwargs)    
    ok = resp['ResponseMetadata']['HTTPStatusCode']==200
    return ok, resp

Here is a non-automated test that engages the live aws service.

In [14]:
import uuid
def LIVE_test():
    client = make_local_client()
    
    secret_name = 'unittest/{}'.format(uuid.uuid4())

    #get an secret before its created
    with TESTCASE.assertRaises(client.exceptions.ResourceNotFoundException):
        get(secret_name, client=client)
    TESTCASE.assertIsNone(get(secret_name, client=client, fail_safely=True))
    ok, resp = create(secret_name, 'testvalue', client=client)
    #print(ok, resp)
    TESTCASE.assertTrue(ok)

    TESTCASE.assertEqual( get(secret_name, client=client), 'testvalue')

    ok, resp = update(secret_name, 'newvalue', client=client)
    #print(ok, resp)
    TESTCASE.assertTrue(ok)
    TESTCASE.assertEqual( get(secret_name, client=client), 'newvalue')

    ok, resp = delete(secret_name, client=client)
    #print(ok, resp)

    with TESTCASE.assertRaises(CLIENT.exceptions.InvalidRequestException):
        get(secret_name, client=client)
        
LIVE_test()

In [15]:
ls_container.stop()

In [16]:
notebook2script(_nbpath)

Converted secret.ipynb.
